In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet("./data/clean_data_merged_v1.parquet")
data['HomeTeamName'] = data['HomeTeamName'].str.strip()
data['AwayTeamName'] = data['AwayTeamName'].str.strip()

In [3]:
data.head(5)

,date,HomeTeamName,AwayTeamName,HomeTeamGoals,AwayTeamGoals,y_classif,y_reg
0,1960-07-06 20:00:00,France,Yugoslavia,4,5,-1.0,-1
1,1960-07-06 21:30:00,Czechoslovakia,Soviet Union,0,3,-1.0,-3
2,1960-07-09 21:30:00,Czechoslovakia,France,2,0,1.0,2
3,1960-07-10 21:30:00,Soviet Union,Yugoslavia,2,1,1.0,1
4,1964-06-17 20:00:00,Spain,Hungary,2,1,1.0,1


In [4]:
def baseline_wins(df):
    home = df.groupby("HomeTeamName")['y_classif'].mean()
    away = df.groupby("AwayTeamName")['y_classif'].mean() * -1
    
    return 0.5*home + 0.5*away

In [5]:
from sklearn.metrics import classification_report, roc_auc_score

In [9]:
for year in [2004, 2008, 2012, 2016]:
    tr = data[data['date'].dt.year < year].copy()
    val = data[data['date'].dt.year == year].copy()
    #print(year, tr.shape, val.shape)
    
    baseline = baseline_wins(tr)
    val['p'] = (val['HomeTeamName'].map(baseline) > val['AwayTeamName'].map(baseline)).astype(int)
    val['p'] = val['p'].map(lambda x: -1 if x == 0 else 1 )
    
    print(year,"Baseline Historical wins", (val['y_classif'] == val['p']).mean())
    print(year, "Baseline Home Win", (val['y_classif'] == np.ones(val.shape[0])).mean())
    print(classification_report(val['y_classif'], val['p']))
    #print(classification_report(val['y_classif'], np.ones(val.shape[0])))
    
    print(year,"AUC Baseline Historical wins", roc_auc_score(val['y_classif'], val['p'], multi_class='ovr'))
    print()
    
    
    

2004 Baseline Historical wins 0.25806451612903225
2004 Baseline Home Win 0.3548387096774194
              precision    recall  f1-score   support

        -1.0       0.19      0.30      0.23        10
         0.0       0.00      0.00      0.00        10
         1.0       0.33      0.45      0.38        11

    accuracy                           0.26        31
   macro avg       0.17      0.25      0.21        31
weighted avg       0.18      0.26      0.21        31



/Users/mariofilho/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mariofilho/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mariofilho/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


AxisError: axis 1 is out of bounds for array of dimension 1